## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
from dotenv import load_dotenv
from langchain.chains.llm import LLMChain
from sympy.physics.units import temperature

load_dotenv()

True

## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

> - 왜 질문이라고 하지 않고 Prompt로 하는가? 물어보는 용도가 아니라 llm에게 지시한다는 의미이다.
> - ChatGPT에서는 완성된 문장으로 물어보지만, 우리가 만드는 것은 특정 템플릿에 내용을 변경하여 요청하는 구조를 가진다. 즉, 변수를 지정하여 질문할 때 치환하는 구조를 가진다.

In [1]:
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [13]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 100자로 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 예전 사용 방식
# from langchain import LLMChain
# chain = LLMChain(prompt=prompt, llm=model)
chain = prompt | model



### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [8]:
input = {"topic": "python의 장점"}

In [9]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='파이썬(Python)은 많은 개발자와 데이터 과학자들이 선호하는 프로그래밍 언어입니다. 그 이유는 다음과 같은 여러 가지 장점이 있기 때문입니다:\n\n1. **쉬운 문법**: 파이썬은 문법이 간단하고 읽기 쉬워서 초보자도 쉽게 배울 수 있습니다. 코드가 명확하고 직관적이어서 다른 사람의 코드를 이해하기도 쉽습니다.\n\n2. **다양한 용도**: 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 스크립트 등 다양한 분야에서 사용됩니다. 하나의 언어로 여러 가지 작업을 수행할 수 있습니다.\n\n3. **강력한 라이브러리**: 파이썬은 다양한 라이브러리와 프레임워크를 제공합니다. 예를 들어, 데이터 분석을 위한 Pandas, 머신러닝을 위한 TensorFlow와 scikit-learn, 웹 개발을 위한 Django와 Flask 등이 있습니다.\n\n4. **활발한 커뮤니티**: 파이썬은 전 세계적으로 많은 사용자와 개발자들이 있어, 문제를 해결하거나 도움을 받을 수 있는 커뮤니티가 활발합니다. 다양한 자료와 튜토리얼도 쉽게 찾을 수 있습니다.\n\n5. **플랫폼 독립성**: 파이썬은 Windows, macOS, Linux 등 다양한 운영체제에서 실행할 수 있어, 개발 환경에 구애받지 않습니다.\n\n6. **빠른 프로토타이핑**: 파이썬은 코드 작성이 빠르고, 실행 속도도 빠르기 때문에 아이디어를 신속하게 구현하고 테스트할 수 있습니다.\n\n이러한 장점들 덕분에 파이썬은 많은 사람들에게 사랑받는 언어가 되었습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 383, 'prompt_tokens': 17, 'total_tokens': 400, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reaso

아래는 스트리밍을 출력하는 예시 입니다.

In [15]:
from share.messages import stream_response

answer = chain.stream(input)
stream_response(answer)


Python은 간결하고 읽기 쉬운 문법으로 배우기 쉽고, 다양한 라이브러리와 프레임워크를 통해 데이터 분석, 웹 개발, 인공지능 등 다양한 분야에 활용됩니다. 또한, 커뮤니티가 활발하여 지원이 풍부합니다.

In [17]:
# 변수가 1개인 경우는 dict으로 전달하지 않아도 된다.
answer = chain.invoke("한국의 수도는?")
print(answer)

content='한국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화 중심지로, 인구가 약 1000만 명에 달합니다. 역사적으로 중요한 유적지와 현대적인 건축물이 공존하며, 교통이 발달해 있습니다. 서울은 세계적인 도시로 성장하고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 20, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-6e086c4d-3066-44be-a387-c37471b0039e-0' usage_metadata={'input_tokens': 20, 'output_tokens': 68, 'total_tokens': 88, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 출력파서(Output Parser)


In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [20]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [21]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

AIMessage(content='인공지능 모델은 대량의 데이터를 기반으로 패턴을 학습합니다. 입력 데이터와 정답을 비교해 오차를 줄이는 방향으로 가중치를 조정하며, 반복 학습을 통해 성능을 향상시킵니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 24, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd14bb27-714c-4847-862a-697174293127-0', usage_metadata={'input_tokens': 24, 'output_tokens': 56, 'total_tokens': 80, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [22]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [23]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [25]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. Can I take your order?  
  **You:** Yes, I'd like to start with a Caesar salad, please.  
  **Waiter:** Great choice! And for your main course?  
  **You:** I’ll have the grilled salmon with vegetables.  
  **Waiter:** Excellent! Would you like anything to drink?  
  **You:** Yes, I'll have a glass of white wine, please.  
  **Waiter:** Perfect! I’ll be right back with your order.  

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 주문 도와드릴까요?  
  **당신:** 네, 먼저 시저 샐러드 하나 주세요.  
  **웨이터:** 훌륭한 선택입니다! 메인 코스는 무엇으로 하시겠습니까?  
  **당신:** 구운 연어와 야채로 할게요.  
  **웨이터:** 훌륭합니다! 음료는 무엇을 드릴까요?  
  **당신:** 네, 화이트 와인 한 잔 주세요.  
  **웨이터:** 완벽합니다! 주문하신 대로 바로 가져오겠습니다.


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)